# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: Diego Orozco Alvarado

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("local[*]") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 02:35:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
import subprocess
import textwrap

LOG_PATH = "/opt/spark/work-dir/data/custom-logs"

# crear logs (segundo plano)
code = textwrap.dedent(f"""
    from diego_orozco.custom_logs import customLogs
    clogs = customLogs()
    clogs.create_log_files(r"{LOG_PATH}", 50)
""")
process = subprocess.Popen(
    ["python3", "-c", code],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col

spark = SparkSession.builder.appName("LogStreaming").getOrCreate()

source_logs = spark.readStream \
    .format("text") \
    .load(LOG_PATH)

# [yyy-mm-dd 00:00:00] level: msg
pattern = r"\[(.*?)\]\s+(\w+):\s+(.*)"
parsed_logs = source_logs \
    .withColumn("date", regexp_extract(col("value"), pattern, 1)) \
    .withColumn("level", regexp_extract(col("value"), pattern, 2)) \
    .withColumn("message", regexp_extract(col("value"), pattern, 3)) \
    .select("date", "level", "message")

query = parsed_logs.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination(100)


-------------------------------------------
Batch: 3
-------------------------------------------
-------------------------------------------
Batch: 8
-------------------------------------------


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+-------+-----------------------------------+
|date               |level  |message                            |
+-------------------+-------+-----------------------------------+
|2025-10-10 03:02:06|WARNING|WARNING in Updating Data           |
|2025-10-10 03:02:06|INFO   |Unauthorized succesfully           |
|2025-10-10 03:02:06|DEBUG  |Connection to DB                   |
|2025-10-10 03:02:06|DEBUG  |Memory usage high                  |
|2025-10-10 03:02:06|DEBUG  |Invalid data                       |
|2025-10-10 03:02:06|ERROR  |ERROR in New user registered       |
|2025-10-10 03:02:06|INFO   |Memory usage high succesfully      |
|2025-10-10 03:02:06|WARNING|WARNING in Retriving Data          |
|2025-10-10 03:02:06|INFO   |New user registered succesfully    |
|2025-10-10 03:02:06|ERROR  |ERROR in Connection to DB          |
|2025-10-10 03:02:06|INFO   |Updating Data su

False